# Report

## 요약



---


Qwen1.5-0.5B 모델을 활용해 "Unreal5 C++ 코드 생성기" 모델을 구현하였다. 학습 방식은 SFT, GAN & PPO 가 사용되었다. PPO 시에 발생하는 메모리 문제를 해결하기 위해 Flash Attention, LoRA, Quantization(8bit) 을 적용하였다. 이후 추가로 발생하는 메모리 문제에 대해서는 Partial PPO 로 대응하였다. 그 결과 코드 특화 모델이 아닌 Qwen1.5-0.5B 를 코드 생성기로 튜닝하는 것에 성공하였다.


## 서론


---


Unreal Engine 5는 자신만의 C++ 문법을 사용하고 있다. 따라서 C++ STL 과 다른 언리얼 만의 매크로, 코드 포맷을 정확히 숙지해야 한다. 이번 과제에서는 Qwen1.5-0.5B 처럼 작은 모델도 이러한 문법을 학습할 수 있는지 실험해보고자 하였다.

학습에는 Supervised Fine-Tuning(SFT), Generative Adversarial Network(GAN), Proximal Policy Optimization(PPO) 등의 기법이 적용되었다.


## 방법


---


본 과제에서 사용된 학습 파이프라인은 두 가지 주요 단계로 구성된다: SFT, GAN & PPO 이다.

1. SFT

  초기 단계에서는 Unreal Engine 관련 C++ 헤더 및 구현 파일을 포함한 데이터셋을 사용해 Qwen1.5-0.5B를 감독 학습 방식으로 튜닝하였다. 이 과정은 기본적인 문법, 스타일, Unreal 전용 키워드(UCLASS, GENERATED_BODY 등)의 사용 습관을 모델에 학습시키는 것을 목표로 했다.

  해당 학습은 1160 개의 데이터를 batch_size 8 로 3 epoch 학습시켰다. 한 epoch 당 소요 시간은 약 4분이었다.

  https://colab.research.google.com/drive/1qLR4PgoErmPj2PCgydMt7dmdFaooByIx#scrollTo=ES4t0TPwQCvi

2. GAN & PPO

  SFT 이후, Generator로서의 Qwen 모델과 Discriminator 모델을 구성하여 GAN & PPO 훈련을 수행하였다. Generator는 Unreal 코드 샘플을 생성하고, Discriminator는 그것이 GPT-4o 로 생성한 것인지 여부를 판단한다. 판단 결과는 PPO 로직을 통해 Generator 를 학습시킨다.
  
  보상 함수는 코드 포맷 준수 패널티와 Discriminator 의 판단 결과로 구성되었다. 이 과정에서 발생하는 VRAM 과부하 문제를 해결하기 위해 FlashAttention과 8bit 양자화, LoRA를 활용하였다. 또한 Gradient Memory를 줄이기 위해 Partial PPO를 도입하였다.

  여기서 말하는 Partial PPO 란, PPO 를 위한 토큰의 기울기 저장을, 전체 토큰이 아닌 일부 토큰에서만 진행한다는 것이다. 예를 들어 총 512 개의 생성 토큰 중 앞 320 개의 토큰과, 나머지 토큰중 10% 의 토큰에 대해서만 기울기를 저장한다. 모든 토큰의 기울기를 저장하는 것에 비해 학습의 수렴 속도가 늦어지겠지만, 학습시 VRAM 피크를 크게 낮출 수 있다. 이 덕분에 기존에는 불가능 하던 PPO 학습을 코랩 환경에서 진행시킬 수 있었다.

  해당 학습은 1160 개의 데이터를 batch_size 2 로 0.2 epoch 학습시켰다. 한 epoch 당 소요 시간은 약 41시간 이었다.


## 모델 구조


---


Generator : Qwen1.5-0.5B, input 1024 tokens, output 512 tokens

Discriminator : Qwen1.5-0.5B, input : 512 tokens, output : 1 logit

Value Head : microsoft/codebert-base


## 실험


---


모델의 성능 평가를 위해 90 개의 requirement, header_code, cpp_code 를 추가로 생성했다. 평가 지표는 아래와 같다.

1. Discriminator Score
  
    훈련된 Discriminator가 판단한 ‘GPT-4o 같은 코드’확률을, 학습 안한 상태를 0점, GPT-4o 를 100점으로 두고 환산한 결과.

2. BLEU Score

    생성된 코드가 정답 코드와 얼마나 유사한지를 측정해 0점에서 100점으로 환산한 결과.


## 결과 및 분석


---


측정 결과는 아래와 같다.

| 모델 단계   | BLEU Score | Discriminator Score |
| ------- | -------- | ------------- |
| Raw | 2.84     | 0.00
| SFT   | 28.16     | 86.11
| SFT+PPO   | **45.06** | **97.59**

SFT+PPO 의 점수가 월등히 높았다. 특히  BLEU Score 에서는 SFT 만 진행한 것 보다 1.5배 이상 높은 수치를 보여준다.


## 결론


---


이번 과제에서는 Qwen1.5-0.5B와 강화학습 기반의 튜닝 기법을 결합하여 Unreal Engine 5 C++ 코드 생성기를 구현하였다. SFT, GAN & PPO의 학습을 통해 모델은 Unreal 특화 문법과 코드 구조를 학습하였다.

향후에는 1024 input 토큰의 여유분을 이용해 RAG 나 프롬프트를 강화할 수 있고, 자동 오류 감지 및 수정 등 다양한 방향으로 응용 가능할 것으로 보인다.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6

/content/drive/MyDrive/Colab Notebooks/Github/AIPlus99/HW6


In [ ]:
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

In [ ]:
# PPO-based training loop for CodeGenerator
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
from torch import nn
from copy import deepcopy
from tqdm import tqdm
from code_generator_1_3b import CodeGenerator
from code_discriminator import CodeDiscriminator
from requirement_to_code_dataset import get_train_dataloader
from value_head import ValueHead
from bitsandbytes.optim import AdamW8bit
import gc

def batched_discriminator(discriminator, texts, batch_size=1):
    preds = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        logits = discriminator(chunk)
        preds.append(logits.cpu())
    return torch.cat(preds, dim=0).to(discriminator.classifier[0].weight.device)

FORMAT_PENALTY_WEIGHT_H = 1.0 / 1.0
FORMAT_PENALTY_WEIGHT_CPP = 1.0 / 1.0

def compute_format_penalties_h(header_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -1.0 if ("UCLASS" not in text and "USTRUCT" not in text and "UINTERFACE" not in text) else 0.0,
            -1.0 if "#pragma once" not in text else 0.0,
            -1.0 if '#include "CoreMinimal.h"' not in text else 0.0,
            -1.0 if '.generated.h' not in text else 0.0,
            -1.0 if 'GENERATED_BODY()' not in text else 0.0,
            -1.0 if '_API' not in text and ("UCLASS" in text or "UINTERFACE" in text) else 0.0,
        ])
        for text in header_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_H * raw_penalties

def compute_format_penalties_cpp(cpp_texts):
    raw_penalties = torch.tensor([
        sum([
            -1.0 if not text.strip().startswith("```cpp") else 0.0,
            -2.0 if not text.strip().endswith("```") else 0.0,
            -6.0 if "::" not in text else 0.0,
        ])
        for text in cpp_texts
    ], dtype=torch.float32)
    return FORMAT_PENALTY_WEIGHT_CPP * raw_penalties

def compute_log_ratios(log_probs, ref_log_probs, clip=10):
    log_ratio = (log_probs - ref_log_probs).clamp(min=-clip, max=clip)
    return torch.exp(log_ratio)

def compute_advantages(rewards, values, normalize=False):
    adv = rewards - values.detach()
    if normalize:
        return (adv - adv.mean()) / (adv.std() + 1e-6)
    else:
        return adv

def compute_ppo_loss(ratios, advantages, clip_eps):
    ratios = ratios.squeeze()
    advantages = advantages.squeeze()
    clipped = torch.clamp(ratios, 1 - clip_eps, 1 + clip_eps)

    surrogate1 = ratios * advantages
    surrogate2 = clipped * advantages

    surrogate = torch.where(
        advantages >= 0,
        torch.min(surrogate1, surrogate2),
        torch.max(surrogate1, surrogate2)
    )

    return -torch.mean(surrogate)

def compute_value_loss(values_disc, reward_disc, values_format, reward_format):
    loss_disc = nn.functional.mse_loss(values_disc, reward_disc.to(values_disc.dtype))
    loss_format = nn.functional.mse_loss(values_format, reward_format.to(values_format.dtype))
    return loss_disc + loss_format

def compute_sft_loss(generator, tokenizer, prompts, targets):
    joined_inputs = [p + t for p, t in zip(prompts, targets)]
    inputs = tokenizer(joined_inputs, return_tensors="pt", padding=True, truncation=True,
                       max_length=generator.max_length).to(generator.device)
    labels = inputs["input_ids"].clone()

    prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True,
                              max_length=generator.max_length)
    prompt_lens = [len(p[p != tokenizer.pad_token_id]) for p in prompt_inputs["input_ids"]]
    for i, l in enumerate(prompt_lens):
        labels[i, :l] = -100

    output = generator(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    return output.loss


def update_generator(total_loss, generator, g_optimizer):
    total_loss.backward()
    torch.nn.utils.clip_grad_norm_(generator.parameters(), 1.0)
    g_optimizer.step()
    g_optimizer.zero_grad()

def update_discriminator(discriminator, d_optimizer, reference_texts, generated_texts, device):
    all_texts = reference_texts + generated_texts
    labels = torch.tensor([1] * len(reference_texts) + [0] * len(generated_texts), dtype=torch.float32).to(device)
    preds = batched_discriminator(discriminator, all_texts)
    loss = nn.BCEWithLogitsLoss()(preds, labels)

    d_optimizer.zero_grad()
    loss.backward()
    d_optimizer.step()


def train(generator, train_loader, epochs=3, lr=2e-5, clip_eps=0.2, do_PPO=True):
    generator.train()

    discriminator = CodeDiscriminator(load_path="./checkpoint_1_3b/discriminator").to(device)

    discriminator.train()

    ref_generator = CodeGenerator()

    ref_generator = deepcopy(generator)
    ref_generator.eval()
    for param in ref_generator.parameters():
        param.requires_grad = False

    g_optimizer = AdamW8bit(generator.parameters(), lr=lr)

    d_optimizer = AdamW8bit(discriminator.parameters(), lr=1e-4)

    os.makedirs("./checkpoint_1_3b", exist_ok=True)

    sft_coef = 0.5
    value_coef = 4.0

    do_SFT_h = False
    do_SFT_cpp = False

    if do_PPO == False:
        do_SFT_h = True
        do_SFT_cpp = True

    for epoch in range(epochs):
        loop = tqdm(train_loader)

        for step, batch in enumerate(loop):
            torch.cuda.empty_cache()

            requirements = batch["requirement"]
            reference_headers = batch["header_code"]
            reference_cpps = batch["cpp_code"]

            # === [Phase 1] Header PPO Sampling ===

            header_prompts = [generator.create_short_prompt_header(r) for r in requirements]

            # === [Phase 1-A] SFT Loss & Step ===
            if do_SFT_h:
                sft_loss = compute_sft_loss(generator, generator.tokenizer, header_prompts, reference_headers)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                g_optimizer.zero_grad()
                sft_loss.backward()
                g_optimizer.step()

                # Cleanup SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()

            if do_PPO == True:
                # === [Phase 1-B] PPO Loss ===
                header_output = generator.sample_header_with_partial_grad(requirements, max_track_tokens=512)
                print("\n===== .h =====")
                print(header_output["header_texts"][0])
                print(header_output["header_texts"][1])

                header_texts = header_output["header_texts"]

                with torch.no_grad():
                    ref_header_output = ref_generator.sample_header_with_partial_grad(requirements, max_track_tokens=512)

                with torch.no_grad():
                    d_scores = torch.sigmoid(discriminator(header_texts)).squeeze()
                    reward_disc = 2 * (d_scores - 0.5)
                    format_penalties = compute_format_penalties_h(header_texts).to(generator.device)
                    if format_penalties.mean().item() < -0.01:
                        print("format_penalties < -0.01")
                        do_SFT_h = True
                    else:
                        do_SFT_h = False
                    header_rewards = reward_disc + format_penalties

                log_probs     = header_output["header_log_probs"].mean(dim=1)
                ref_log_probs = ref_header_output["header_log_probs"].mean(dim=1)
                ratios        = compute_log_ratios(log_probs, ref_log_probs)

                values_total, values_disc, values_format = generator.compute_value(header_prompts, header_texts, mode="h")
                advantages = compute_advantages(header_rewards, values_total)

                print(f"reward_disc : {reward_disc}")
                print(f"values_disc : {values_disc}")

                print(f"format_penalties : {format_penalties}")
                print(f"values_format : {values_format}")

                print(f"ratios : {ratios}")
                print(f"advantages : {advantages}")
                ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
                value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
                total_loss = ppo_loss + value_coef * value_loss

                # PPO update
                g_optimizer.zero_grad()
                total_loss.backward()
                g_optimizer.step()

                # === Discriminator training ===
                if reward_disc.mean().item() > -0.4 or step % 10 == 0:
                    print(f"\n[Step {step}] Training header discriminator...")
                    update_discriminator(discriminator, d_optimizer, reference_headers, header_texts, generator.device)

                # === Logging ===
                loop.set_description(f"[Epoch {epoch+1}] Header")
                loop.set_postfix({
                    "ppo":    ppo_loss.item(),
                    "value":  value_loss.item(),
                    "reward": header_rewards.mean().item()
                })

                # === Cleanup ===
                for var_name in [
                    "log_probs", "ref_log_probs", "ratios",
                    "values_total", "values_disc", "values_format",
                    "d_scores", "reward_disc", "format_penalties", "header_rewards",
                    "advantages", "ppo_loss", "value_loss", "total_loss"
                ]:
                    var = locals().get(var_name)
                    if isinstance(var, torch.Tensor):
                        locals()[var_name] = var.detach()
                    del var

                del header_output, ref_header_output, header_texts, header_prompts
                torch.cuda.empty_cache()


            # === [Phase 2] CPP PPO 학습 ===

            cpp_prompts = [generator.create_short_prompt_cpp(r, h) for r, h in zip(requirements, reference_headers)]

            # === [Phase 2-A] SFT Loss & Step ===
            if do_SFT_cpp:
                sft_loss = compute_sft_loss(generator, generator.tokenizer, cpp_prompts, reference_cpps)
                print(f"\nsft_loss : {sft_loss}")
                sft_loss = sft_loss * sft_coef
                g_optimizer.zero_grad()
                sft_loss.backward()
                g_optimizer.step()

                # Clean up SFT memory
                sft_loss = sft_loss.detach()
                del sft_loss
                torch.cuda.empty_cache()
                gc.collect()


            if do_PPO == True:
                # === [Phase 2-B] PPO + Value Loss ===

                cpp_output = generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=320)
                print("\n===== .cpp =====")
                print(cpp_output["cpp_texts"][0])
                print(cpp_output["cpp_texts"][1])

                cpp_texts = cpp_output["cpp_texts"]

                with torch.no_grad():
                    ref_cpp_output = ref_generator.sample_cpp_with_partial_grad(requirements, reference_headers, max_track_tokens=320)

                with torch.no_grad():
                    d_scores = torch.sigmoid(discriminator(cpp_texts)).squeeze()
                    reward_disc = 2 * (d_scores - 0.5)
                    format_penalties = compute_format_penalties_cpp(cpp_texts).to(generator.device)
                    if format_penalties.mean().item() < -0.01:
                        print("format_penalties < -0.01")
                        do_SFT_cpp = True
                    else:
                        do_SFT_cpp = False
                    cpp_rewards = reward_disc + format_penalties

                log_probs     = cpp_output["cpp_log_probs"].mean(dim=1)
                ref_log_probs = ref_cpp_output["cpp_log_probs"].mean(dim=1)
                ratios        = compute_log_ratios(log_probs, ref_log_probs)

                values_total, values_disc, values_format = generator.compute_value(cpp_prompts, cpp_texts, mode="cpp")
                advantages = compute_advantages(cpp_rewards, values_total)

                print(f"reward_disc : {reward_disc}")
                print(f"values_disc : {values_disc}")

                print(f"format_penalties : {format_penalties}")
                print(f"values_format : {values_format}")

                print(f"ratios : {ratios}")
                print(f"advantages : {advantages}")
                ppo_loss   = compute_ppo_loss(ratios, advantages, clip_eps)
                value_loss = compute_value_loss(values_disc, reward_disc, values_format, format_penalties)
                total_loss = ppo_loss + value_coef * value_loss

                # PPO step
                g_optimizer.zero_grad()
                total_loss.backward()
                g_optimizer.step()

                # === Train Discriminator on CPP ===
                if reward_disc.mean().item() > -0.4 or step % 10 == 0:
                    print(f"\n[Step {step}] Training cpp discriminator...")
                    update_discriminator(discriminator, d_optimizer, reference_cpps, cpp_texts, generator.device)

                # === Logging ===
                loop.set_description(f"[Epoch {epoch+1}] CPP")
                loop.set_postfix({
                    "ppo":    ppo_loss.item(),
                    "value":  value_loss.item(),
                    "reward": cpp_rewards.mean().item()
                })

                # === Cleanup ===
                for var_name in [
                    "log_probs", "ref_log_probs", "ratios",
                    "values_total", "values_disc", "values_format",
                    "d_scores", "reward_disc", "format_penalties", "cpp_rewards",
                    "advantages", "ppo_loss", "value_loss", "total_loss"
                ]:
                    var = locals().get(var_name)
                    if isinstance(var, torch.Tensor):
                        locals()[var_name] = var.detach()
                    del var

                del cpp_output, ref_cpp_output, cpp_texts, cpp_prompts

                torch.cuda.empty_cache()


            # === 주기적 ref_generator 동기화 ===
            if step % 10 == 0:
                print(f"\n[Step {step}] Ref generator updated.")
                del ref_generator

                ref_generator = deepcopy(generator)
                ref_generator.eval()
                ref_generator = ref_generator.to(generator.device)
                for p in ref_generator.parameters():
                    p.requires_grad = False

                with torch.no_grad():
                    h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],  max_track_tokens=0)
                    cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"], max_track_tokens=0)

                print("\n===== .h =====")
                print(h_generated_responses["header_texts"][0])
                print("\n===== .cpp =====")
                print(cpp_generated_responses["cpp_texts"][0])

                # Save checkpoint
                generator.save("./checkpoint_1_3b/generator")
                discriminator.save("./checkpoint_1_3b/discriminator")

        with torch.no_grad():
            h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],max_track_tokens=0)
            cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"],max_track_tokens=0)
        print("\n===== .h =====")
        print(h_generated_responses["header_texts"][0])
        print("\n===== .cpp =====")
        print(cpp_generated_responses["cpp_texts"][0])


if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    save_path = "./checkpoint_1_3b"

    generator = CodeGenerator(load_path="./checkpoint_1_3b/generator")

    """
    with torch.no_grad():
        h_generated_responses = generator.sample_header_with_partial_grad(["Create a character class with health and mana properties"],  max_track_tokens=0)
        cpp_generated_responses = generator.sample_cpp_with_partial_grad(["Create a character class with health and mana properties"], h_generated_responses["header_texts"], max_track_tokens=0)
    print("\n===== .h =====")
    print(h_generated_responses["header_texts"][0])
    print("\n===== .cpp =====")
    print(cpp_generated_responses["cpp_texts"][0])
    """

    train_loader = get_train_dataloader("unreal_code_dataset.jsonl", batch_size=2, shuffle=True, limit= 1160)

    train(generator, train_loader, epochs=5, do_PPO=True)



Loading model + LoRA from ./checkpoint_1_3b/generator


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Freezing all parameters except LoRA blocks...
Trainable params: 1,572,864 / 465,560,576 (0.34%)
Loaded value head.


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading discriminator from ./checkpoint_1_3b/discriminator
Loaded classifier head from ./checkpoint_1_3b/discriminator/classifier.pt
Initializing new model from base: Qwen/Qwen1.5-0.5B
Applying LoRA adaptation...
Freezing all parameters except LoRA blocks...
Trainable params: 1,572,864 / 465,560,576 (0.34%)
No value head to load


...

[Epoch 1] Header:  20%|██        | 116/580 [8:26:51<38:07:31, 295.80s/it, ppo=-0.233, value=0.0469, reward=-0.0969]


===== .cpp =====
```cpp
#include "HealthRegenSystem.h"
#include "GameFramework/Actor.h"

AHealthRegenSystem::AHealthRegenSystem()
{
	PrimaryComponent::FindWorld()->AddDynamicComponent(new FRegenHealthComponent(this, true));
	MaxHealth = 100.0f;
	CurrentHealth = 0.0f;
	RegenRate = 1.0f;
}

void AHealthRegenSystem::BeginPlay()
{
	Super::BeginPlay();
	RegenerateHealth(0.0f);
}

void AHealthRegenSystem::Tick(float DeltaTime)
{
	Super::Tick(DeltaTime);

	if (CurrentHealth > MaxHealth)
	{
		CurrentHealth = 0.0f;
		RegenerateHealth(0.0f);
	}
}

void AHealthRegenSystem::RegenerateHealth(float DeltaTime)
{
	if (CurrentHealth > 0.0f)
	{
		CurrentHealth = CurrentHealth + (1.0f * RegenRate * DeltaTime);
	}
}
```
```cpp
#include "ULivesWidget.h"
#include "TextBlock.h"

ULivesWidget::ULivesWidget(const FObjectInitializer& ObjectInitializer)
{
	PrimaryComponentTick.bCanEverTick = false;

	LivesText = CreateTextBlock(0, "Lives: ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[Epoch 1] CPP:  20%|██        | 117/580 [8:31:25<33:43:50, 262.27s/it, ppo=0.844, value=1.51, reward=-1.09]


===== .h =====
```cpp
#pragma once

#include "CoreMinimal.h"
#include "GameFramework/Actor.h"
#include "ToggleOnOffActor.generated.h"

UCLASS()
class YOURPROJECT_API AToggleOnOffActor : public AActor
{
	GENERATED_BODY()

public:
	// Sets default values for this actor's properties
	AToggleOnOffActor();

protected:
	// Called when the game starts or when spawned
	virtual void BeginPlay() override;

public:
	// Called every frame
	virtual void Tick(float DeltaTime) override;

private:
	void ToggleOnOff();

	// Toggle on/off the actor
	void ToggleOnOffFunction();

	// Function to toggle the actor on/off
	void ToggleOnOffFunctionCallback();
};
```
```cpp
#pragma once

#include "CoreMinimal.h"
#include "GameFramework/Actor.h"
#include "TimeChangingMaterial.generated.h"

UCLASS()
class YOURPROJECT_API ATimeChangingMaterial : public AActor
{
	GENERATED_BODY()

public:
	// Sets default values for this actor's properties
	ATimeChangingMaterial();

protected:
	// Called when the game starts or w

KeyboardInterrupt: 

In [ ]:
generator.save("./checkpoint_1_3b/generator")